In [1]:
from scipy.stats import poisson
import matplotlib.pyplot as plt
import pyodbc
import pandas as pd
import numpy as np
FinalData = pd.DataFrame([])
test = pd.DataFrame([])
foo = pd.DataFrame([])

In [2]:
Cust_Data_SQL = """ \
        SELECT DISTINCT
        InvoiceDate,
        CustomerAddressKey,
        dd.CurrentDayOffset
        INTO ##CustData
        FROM [USVBIAnalytics].[DWAF].[vwFactSales] fs
        LEFT JOIN [USVBIAnalytics].[DW].[vwDimDate] dd ON fs.InvoiceDateKey = dd.DateKey
        WHERE CustomerAddressKey > 0 AND dd.CurrentDayOffset >= -365
        ORDER BY InvoiceDate
"""

Indexed_Data_SQL = """ \
       SELECT ROW_NUMBER() OVER (PARTITION BY CustomerAddressKey ORDER BY InvoiceDate ASC) AS Row#,
       InvoiceDate,
       CustomerAddressKey
INTO ##IndexedData
FROM ##CustData;
"""

Final_Data_SQL = """ \
      SELECT A.InvoiceDate AS Date1,
       B.InvoiceDate AS Date2,
       A.CustomerAddressKey,
       DATEDIFF(DAY, A.InvoiceDate, B.InvoiceDate) AS InnerEventTime,
       A.Row# AS Row1,
       B.Row# AS Row2
       INTO ##FinalData
       FROM ##IndexedData A
       JOIN ##IndexedData B ON A.Row# = B.Row# - 1 AND B.CustomerAddressKey = A.CustomerAddressKey
--       WHERE  A.CustomerAddressKey in ('36646', '36642', '36644')
       ORDER BY 3,5
"""

In [3]:
conn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server}; Server=SQLPDW; Database=USVBIanalytics; Trusted_Connection=yes;')
cursor=conn.cursor()
cursor.execute(Cust_Data_SQL)
cursor.execute(Indexed_Data_SQL)
cursor.execute(Final_Data_SQL)
conn.commit()
query1="Select CustomerAddressKey, Date1, Date2, InnerEventTime from ##FinalData"
data1= pd.read_sql_query(query1,conn)

In [4]:
CustGroup = data1.groupby('CustomerAddressKey')

In [5]:
foo = pd.DataFrame(CustGroup.InnerEventTime.mean())   
test = pd.merge(data1, foo, on='CustomerAddressKey').rename(index=str, columns={'InnerEventTime_y':'AvgInnerEventTime','InnerEventTime_x':'DaySinceLastPurchase'})

In [6]:
FinalData = test.assign(p = 0).rename(index=str, columns={'p':'Probability'})

In [50]:
FinalData.Probability = (1-poisson.cdf(test.DaySinceLastPurchase, test.AvgInnerEventTime, loc=0))
FinalData[FinalData.CustomerAddressKey == 36642]

,CustomerAddressKey,Date1,Date2,DaySinceLastPurchase,AvgInnerEventTime,Probability
93878,36642,2018-10-24,2018-10-28,4,9.105263,0.948485
93879,36642,2019-06-01,2019-06-24,23,9.105263,0.000029
93880,36642,2018-10-30,2018-11-11,12,9.105263,0.132020
93881,36642,2018-11-20,2018-11-27,7,9.105263,0.688286
93882,36642,2018-12-03,2018-12-04,1,9.105263,0.998878
93883,36642,2019-03-14,2019-03-21,7,9.105263,0.688286
93884,36642,2018-08-28,2018-08-30,2,9.105263,0.994273
93885,36642,2019-03-05,2019-03-06,1,9.105263,0.998878
93886,36642,2019-03-13,2019-03-14,1,9.105263,0.998878
93887,36642,2018-10-28,2018-10-29,1,9.105263,0.998878


In [8]:
FinalData['Date2'] = pd.to_datetime(FinalData['Date2'])
FinalData['Date1'] = pd.to_datetime(FinalData['Date1'])
FinalData.dtypes

CustomerAddressKey               int64
Date1                   datetime64[ns]
Date2                   datetime64[ns]
DaySinceLastPurchase             int64
AvgInnerEventTime              float64
Probability                    float64
dtype: object

In [45]:
MostRecentEvent = FinalData.sort_values(by='Date2').groupby('CustomerAddressKey').last()
# a=MostRecentEvent.reset_index()
# a[a.CustomerAddressKey == 66587]
MostRecentEvent

,Date1,Date2,DaySinceLastPurchase,AvgInnerEventTime,Probability
CustomerAddressKey,,,,,
3,2019-06-26,2019-06-27,1,1.338235,3.866618e-01
8,2019-06-04,2019-06-06,2,13.608696,9.998682e-01
12,2019-06-26,2019-06-27,1,1.733333,5.170352e-01
19,2019-06-26,2019-06-27,1,1.144654,3.172837e-01
28,2019-06-12,2019-06-24,12,15.565217,7.765526e-01
31,2019-06-12,2019-06-18,6,15.434783,9.942187e-01
32,2019-06-24,2019-06-26,2,13.769231,9.998852e-01
35,2019-02-04,2019-06-26,142,77.666667,2.089928e-11
36,2019-06-11,2019-06-24,13,11.900000,3.079528e-01


In [10]:
query2="""Drop Table ##FinalData, ##CustData, ##IndexedData"""
cursor.execute(query2)

In [11]:
cursor.close()

In [12]:
# test.isna().sum()
# if (test[test.CustomerAddressKey == 36644].reset_index(drop=True).iloc[0,:].InnerEventTime_y - a.InnerEventTime_y.mean()) > 1e-4:
#     print('Fail')
# else:
#     print('Pass')

In [13]:

# data1_append

In [14]:
# Group = data1.groupby('CustomerAddressKey')
# mu = Group.mean()
# k = 
# mean, var, skew, kurt = poisson.stats(mu, moments='mvsk')
#Avg_IVT

In [15]:
#data1.shape[0]

In [16]:
#data1.iloc[data1.shape[0]-1,:] #[r,c]

In [17]:
#var

In [18]:
#skew

In [19]:
#mean

In [20]:
#whos

In [21]:
# data.dtypes

In [22]:
# data.describe()

In [23]:
# data.replace('', np.nan, inplace=True)

In [24]:
# print(data.isna().sum())
# print(data.shape)

In [25]:
# conn.close()

In [26]:
# type(data)

In [27]:
#####Active Connection
# config = dict(server=   'SQLPDW', # change this to your SQL Server hostname or IP address
#               port=      1433,                    # change this to your SQL Server port number [1433 is the default]
#               database= 'SQLPDW',
#               username= 'user',
#               password= 'pass')

# trusted_conn_str = ('SERVER={server};'     +
#                     'DATABASE={database};' +
#                     'TRUSTED_CONNECTION=yes')

# conn = pyodbc.connect(
#     r'DRIVER={ODBC Driver 13 for SQL Server};' +
#     trusted_conn_str.format(**config)
#     )